# Final Capstone Project

## What constitues an ideal city

# Intro


I will be analyzing the worst and most popular places to live based on the 2019 Best Places to Live Research by US NEWS

I will be using these cities:

Rank #1: Austin, TX
Rank #2: Denver, CO
Rank #3: Colorado Springs, CO
Rank #4: Fayetteville, AR
Rank #5: Des Moines, IA
Rank #125: San Juan, PR
Rank #124: Bakersfield, CA
Rank #123: Stockton, CA
Rank #122: Shreveport, LA
Rank #121: Mobile, AL

# What is the Problem

It is important to use data to predict cities that might become popular in the future. Although the Rankings provided by USNEWS is very helpful in determining current hotspots. It is imperative to be ahead of the curve and purchase a home in a city before the popularity and prices rise. Therefore, using the data gathered from Foursquare, we will identify variables that correspond with city popularity represented by the City Scores, which could potentially help us find cities that will become popular in the future.

# What Data

Im using the Foursquare API to determine the frequency and ratio of venue types in each of the cities to determine whether there are any significant correlation between the venues found in a city and its overall score. We will use a machine learning algorithms to make predictive model that can can predict for us the city's overall score based on their venues I will pick 4 random cities to test this model.

#  How to do the Math

The data we collected from each of our sources came in a variety of forms, from flat numbers to negative numbers to percentages. Before imputing each data point into our equations, we had to standardize it so that it could be compared on an apples-to-apples basis with other data points we evaluated.
In order to balance these metrics, we used a standard deviation (or Z-Score) calculation to determine each data point's differentiation from the data set's mean, and then used the Z-Score to create a standardized U.S. News score using the method outlined below:
Calculating the Z-Score: The Z-Score represents a data point's relation to the mean measurement of the data set. The Z-Score is a negative when the data point is below the mean and positive when it's above the mean; a Z-Score of 0 means it's equal to the mean. To determine the Z-Score for each data point, we calculated the mean of that same data point across all ranked metro areas. We then subtracted the data point from the mean and divided it by the standard deviation. Calculating the T-Score: We used a T-Score calculation to convert the Z-Score to a 0-100 scale by multiplying the Z-Score by 10. To ensure that the mean was equal across all data points, we added our desired scoring mean (between 0 and 10) to the T-Score to create an adjusted T-Score. Calculating the U.S. News Score: To calculate the U.S. News Score, we divided the adjusted T-Score by 10 to create a 0-10 point system across the board that would allow us to accurately apply our methodology.

In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json 
from geopy.geocoders import Nominatim 
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium 
print('Libraries imported.')

ModuleNotFoundError: No module named 'folium'

In [2]:
data =         [['Austin, TX',7.6,1.0,30.267153,-97.7430608],
                ['Denver, CO',7.4,2.0,39.742043,-104.991531],
                ['Colorado Springs, CO',7.4,3.0,38.846127,-104.800644],
                ['Fayetteville, AR',7.3,4.0,36.082157,-94.171852],
                ['Des Moines, IA',7.3,5.0,41.619549,-93.598022],
                ['San Juan, PR',3.2,125.0,18.466333,-66.105721],
                ['Bakersfield, CA',5.3,124.0,35.393528,-119.043732],
                ['Stockton, CA',5.4,123.0,37.961632,-121.275604],
                ['Shreveport, LA',5.4,122.0,32.523659,-93.763504],
                ['Mobile, AL',5.5,121.0, 30.695366,-88.039894]]

df_score = pd.DataFrame(data, columns =['Cities','Score','Rank','Latitude','Longitude'])
df_score

,Cities,Score,Rank,Latitude,Longitude
0,"Austin, TX",7.6,1.0,30.267153,-97.743061
1,"Denver, CO",7.4,2.0,39.742043,-104.991531
2,"Colorado Springs, CO",7.4,3.0,38.846127,-104.800644
3,"Fayetteville, AR",7.3,4.0,36.082157,-94.171852
4,"Des Moines, IA",7.3,5.0,41.619549,-93.598022
5,"San Juan, PR",3.2,125.0,18.466333,-66.105721
6,"Bakersfield, CA",5.3,124.0,35.393528,-119.043732
7,"Stockton, CA",5.4,123.0,37.961632,-121.275604
8,"Shreveport, LA",5.4,122.0,32.523659,-93.763504
9,"Mobile, AL",5.5,121.0,30.695366,-88.039894


In [4]:
CLIENT_ID = 'XSKY2H0EQZ1H4CV4W0B5HJG2DXGFFKKKCIVSPKW1SHRVTROD' # your Foursquare ID
CLIENT_SECRET = '3EDCYVT1AJTUTVPRIKNM1EIPCEHDHODA1XFVDGHCJLWPKSW3' # your Foursquare Secret
VERSION = '20180605'

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: XSKY2H0EQZ1H4CV4W0B5HJG2DXGFFKKKCIVSPKW1SHRVTROD
CLIENT_SECRET:3EDCYVT1AJTUTVPRIKNM1EIPCEHDHODA1XFVDGHCJLWPKSW3


In [5]:
df_score.index

RangeIndex(start=0, stop=10, step=1)

In [6]:
city_latitude = df_score.loc[0, 'Latitude']
city_longitude = df_score.loc[0, 'Longitude']
city_name = df_score.loc[0, 'Citiesprint('Latitude and longitude values of {} are {}, {}.'.format(city_name, 
                                                               city_latitude, 
                                                               city_longitude))


SyntaxError: invalid syntax (<ipython-input-6-bf4aab3de3fb>, line 3)

In [7]:
city_latitude = df_score.loc[0, 'Latitude']
city_longitude = df_score.loc[0, 'Longitude']
city_name = df_score.loc[0, 'Cities']
print('Latitude and longitude values of {} are {}, {}.'.format(city_name, 
                                                               city_latitude, 
                                                               city_longitude))

Latitude and longitude values of Austin, TX are 30.267153, -97.7430608.


In [8]:
LIMIT = 1000
radius = 2000
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, city_latitude, city_longitude, VERSION, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=XSKY2H0EQZ1H4CV4W0B5HJG2DXGFFKKKCIVSPKW1SHRVTROD&client_secret=3EDCYVT1AJTUTVPRIKNM1EIPCEHDHODA1XFVDGHCJLWPKSW3&ll=30.267153,-97.7430608&v=20180605&radius=2000&limit=1000'

In [9]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d61503e8afbe000351814bf'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': '$-$$$$', 'key': 'price'},
    {'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Downtown Austin',
  'headerFullLocation': 'Downtown Austin, Austin',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 231,
  'suggestedBounds': {'ne': {'lat': 30.285153018000017,
    'lng': -97.7222587515462},
   'sw': {'lat': 30.249152981999984, 'lng': -97.76386284845378}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '56c33b5ccd10422b2cb626ab',
       'name': 'JuiceLand',
       'location': {'address': '120 E 4th St',
        'crossStreet': 'Congress',
        'lat': 30.266037164237066,
        'lng': -97.742622

In [10]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [11]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,JuiceLand,Juice Bar,30.266037,-97.742623
1,Houndstooth Coffee,Coffee Shop,30.266194,-97.743025
2,Caroline Restaurant,Hotel,30.268816,-97.742298
3,Truluck's Seafood Steak & Crab,Seafood Restaurant,30.266757,-97.744968
4,Firehouse Hostel and Lounge,Lounge,30.268096,-97.741283


In [12]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


In [15]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        results = requests.get(url).json()["response"]['groups'][0]['items']
venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
nearby_venues.columns = ['Cities', 
                  'City Latitude', 
                  'City Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
return(nearby_venues)

NameError: name 'venues_list' is not defined

In [16]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Cities', 
                  'City Latitude', 
                  'City Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [17]:
city_venues = getNearbyVenues(names=df_score['Cities'],
                                   latitudes=df_score['Latitude'],
                                   longitudes=df_score['Longitude']
                                  )

Austin, TX
Denver, CO
Colorado Springs, CO
Fayetteville, AR
Des Moines, IA
San Juan, PR
Bakersfield, CA
Stockton, CA
Shreveport, LA
Mobile, AL


In [18]:
print(city_venues.shape)
city_venues.head()

(249, 7)


,Cities,City Latitude,City Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Austin, TX",30.267153,-97.743061,JuiceLand,30.266037,-97.742623,Juice Bar
1,"Austin, TX",30.267153,-97.743061,Houndstooth Coffee,30.266194,-97.743025,Coffee Shop
2,"Austin, TX",30.267153,-97.743061,Caroline Restaurant,30.268816,-97.742298,Hotel
3,"Austin, TX",30.267153,-97.743061,Truluck's Seafood Steak & Crab,30.266757,-97.744968,Seafood Restaurant
4,"Austin, TX",30.267153,-97.743061,Firehouse Hostel and Lounge,30.268096,-97.741283,Lounge


In [19]:
city_venues.groupby('Cities').count()


,City Latitude,City Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Cities,,,,,,
"Austin, TX",100,100,100,100,100,100
"Bakersfield, CA",2,2,2,2,2,2
"Colorado Springs, CO",3,3,3,3,3,3
"Denver, CO",72,72,72,72,72,72
"Des Moines, IA",2,2,2,2,2,2
"Fayetteville, AR",5,5,5,5,5,5
"Mobile, AL",25,25,25,25,25,25
"San Juan, PR",17,17,17,17,17,17
"Shreveport, LA",3,3,3,3,3,3


In [20]:
print('There are {} uniques categories.'.format(len(city_venues['Venue Category'].unique())))

There are 101 uniques categories.


In [21]:

city_onehot = pd.get_dummies(city_venues[['Venue Category']], prefix="", prefix_sep="")

city_onehot['Cities'] = city_venues['Cities'] 


fixed_columns = [city_onehot.columns[-1]] + list(city_onehot.columns[:-1])
city_onehot = city_onehot[fixed_columns]

city_onehot.head()

,Cities,American Restaurant,Arcade,Art Museum,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Beer Bar,Bookstore,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Burger Joint,Business Service,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Cosmetics Shop,Cycle Studio,Deli / Bodega,Dessert Shop,Discount Store,Doctor's Office,Donut Shop,Electronics Store,Farmers Market,Fast Food Restaurant,Food Court,Food Truck,Fried Chicken Joint,Gay Bar,Gift Shop,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Historic Site,Hotel,Hotel Bar,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Latin American Restaurant,Lounge,Marijuana Dispensary,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Motorsports Shop,Movie Theater,Moving Target,Music Venue,New American Restaurant,Nightclub,Noodle House,Optical Shop,Outdoor Sculpture,Paella Restaurant,Park,Pet Store,Pharmacy,Pier,Pizza Place,Plaza,Residential Building (Apartment / Condo),Restaurant,Rock Club,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Southern / Soul Food Restaurant,Souvenir Shop,Spa,Speakeasy,Steakhouse,Sushi Restaurant,Taco Place,Tea Room,Theater,Theme Park Ride / Attraction,Turkish Restaurant,University,Yoga Studio
0,"Austin, TX",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Austin, TX",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Austin, TX",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Austin, TX",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Austin, TX",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [22]:
city_grouped = city_onehot.groupby('Cities').mean().reset_index()
city_grouped

,Cities,American Restaurant,Arcade,Art Museum,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Beer Bar,Bookstore,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Burger Joint,Business Service,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Cosmetics Shop,Cycle Studio,Deli / Bodega,Dessert Shop,Discount Store,Doctor's Office,Donut Shop,Electronics Store,Farmers Market,Fast Food Restaurant,Food Court,Food Truck,Fried Chicken Joint,Gay Bar,Gift Shop,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Historic Site,Hotel,Hotel Bar,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Latin American Restaurant,Lounge,Marijuana Dispensary,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Motorsports Shop,Movie Theater,Moving Target,Music Venue,New American Restaurant,Nightclub,Noodle House,Optical Shop,Outdoor Sculpture,Paella Restaurant,Park,Pet Store,Pharmacy,Pier,Pizza Place,Plaza,Residential Building (Apartment / Condo),Restaurant,Rock Club,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Southern / Soul Food Restaurant,Souvenir Shop,Spa,Speakeasy,Steakhouse,Sushi Restaurant,Taco Place,Tea Room,Theater,Theme Park Ride / Attraction,Turkish Restaurant,University,Yoga Studio
0,"Austin, TX",0.020000,0.000000,0.02,0.00,0.000000,0.0,0.01,0.0,0.000000,0.00,0.040000,0.01,0.00,0.000000,0.000000,0.010000,0.030000,0.000000,0.010000,0.010000,0.000000,0.000000,0.00,0.000000,0.010000,0.050000,0.080000,0.000000,0.01,0.000000,0.010000,0.0,0.00,0.01,0.000000,0.000000,0.00,0.01,0.000000,0.01,0.03,0.010000,0.000000,0.02,0.000000,0.010000,0.000000,0.080000,0.020000,0.01,0.030000,0.000000,0.01,0.030000,0.010000,0.050000,0.000000,0.000000,0.01,0.01,0.000000,0.0,0.030000,0.000000,0.020000,0.030000,0.030000,0.000000,0.01,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.010000,0.000000,0.000000,0.020000,0.02,0.01,0.010000,0.000000,0.000000,0.02,0.0,0.000000,0.00,0.00,0.000000,0.000000,0.02,0.030000,0.02,0.01,0.000000,0.010000,0.000000,0.01,0.0,0.01
1,"Bakersfield, CA",0.000000,0.000000,0.00,0.00,0.000000,0.0,0.00,0.0,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.00,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.5,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.0,0.00
2,"Colorado Springs, CO",0.000000,0.000000,0.00,0.00,0.000000,0.0,0.00,0.0,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.00,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.00,0.333333,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.333333,0.00,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.0,0.00
3,"Denver, CO",0.055556,0.013889,0.00,0.00,0.000000,0.0,0.00,0.0,0.013889,0.00,0.027778,0.00,0.00,0.013889,0.013889,0.013889,0.013889,0.000000,0.013889,0.013889,0.013889,0.000000,0.00,0.013889,0.013889,0.013889,0.027778,0.013889,0.00,0.013889,0.013889,0.0

In [23]:
num_top_venues = 5

for cit in city_grouped['Cities']:
    print("----"+cit+"----")
    temp = city_grouped[city_grouped['Cities'] == cit].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Austin, TX----
          venue  freq
0   Coffee Shop  0.08
1         Hotel  0.08
2  Cocktail Bar  0.05
3        Lounge  0.05
4           Bar  0.04


----Bakersfield, CA----
                 venue  freq
0           Shoe Store   0.5
1     Motorsports Shop   0.5
2  American Restaurant   0.0
3          Music Venue   0.0
4             Pharmacy   0.0


----Colorado Springs, CO----
                     venue  freq
0           Massage Studio  0.33
1                Gift Shop  0.33
2         Business Service  0.33
3      American Restaurant  0.00
4  New American Restaurant  0.00


----Denver, CO----
                 venue  freq
0                Hotel  0.14
1  American Restaurant  0.06
2       Sandwich Place  0.04
3          Coffee Shop  0.03
4             Tea Room  0.03


----Des Moines, IA----
                 venue  freq
0           University   0.5
1           Bagel Shop   0.5
2  American Restaurant   0.0
3        Moving Target   0.0
4             Pharmacy   0.0


----Fayetteville, AR----

In [24]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [25]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']
columns = ['Cities']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

city_venues_sorted = pd.DataFrame(columns=columns)
city_venues_sorted['Cities'] = city_grouped['Cities']
for ind in np.arange(city_grouped.shape[0]):
    city_venues_sorted.iloc[ind, 1:] = return_most_common_venues(city_grouped.iloc[ind, :], num_top_venues)

city_venues_sorted


,Cities,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Austin, TX",Coffee Shop,Hotel,Cocktail Bar,Lounge,Bar,Italian Restaurant,Movie Theater,Nightclub,Burger Joint,Gay Bar
1,"Bakersfield, CA",Shoe Store,Motorsports Shop,Yoga Studio,Farmers Market,Coffee Shop,Cosmetics Shop,Cycle Studio,Deli / Bodega,Dessert Shop,Discount Store
2,"Colorado Springs, CO",Massage Studio,Gift Shop,Business Service,Yoga Studio,Fast Food Restaurant,Cosmetics Shop,Cycle Studio,Deli / Bodega,Dessert Shop,Discount Store
3,"Denver, CO",Hotel,American Restaurant,Sandwich Place,Bar,Coffee Shop,Food Truck,Gym / Fitness Center,Lounge,Mexican Restaurant,Pizza Place
4,"Des Moines, IA",University,Bagel Shop,Food Court,Cosmetics Shop,Cycle Studio,Deli / Bodega,Dessert Shop,Discount Store,Doctor's Office,Donut Shop
5,"Fayetteville, AR",Pizza Place,Automotive Shop,Discount Store,Mexican Restaurant,Bookstore,Grocery Store,Farmers Market,Cosmetics Shop,Cycle Studio,Deli / Bodega
6,"Mobile, AL",Sandwich Place,Bank,Seafood Restaurant,Hotel,Gym / Fitness Center,Asian Restaurant,Hotel Bar,Coffee Shop,Pizza Place,Plaza
7,"San Juan, PR",Plaza,Bar,Farmers Market,Historic Site,Sandwich Place,Caribbean Restaurant,Latin American Restaurant,Pier,Bakery,Music Venue
8,"Shreveport, LA",Business Service,Electronics Store,Moving Target,Yoga Studio,Cocktail Bar,Cosmetics Shop,Cycle Studio,Deli / Bodega,Dessert Shop,Discount Store
9,"Stockton, CA",Fast Food Restaurant,Italian Restaurant,Seafood Restaurant,Grocery Store,Japanese Restaurant,Donut Shop,Mexican Restaurant,Doctor's Office,Deli / Bodega,Chinese Restaurant


In [26]:
for ind in np.arange(city_grouped.shape[0]):
    city_venues_sorted.iloc[ind, 1:] = return_most_common_venues(city_grouped.iloc[ind, :], num_top_venues)

city_venues_sorted
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(city_grouped_clustering)
kmeans.labels_[0:10]

NameError: name 'kclusters' is not defined

In [27]:
kclusters = 6
city_grouped_clustering = city_grouped.drop('Cities', 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(city_grouped_clustering)
kmeans.labels_[0:10]

array([3, 4, 5, 3, 2, 0, 3, 3, 1, 3], dtype=int32)

In [28]:
city_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

city_merged = df_score

city_merged = city_merged.join(city_venues_sorted.set_index('Cities'), on='Cities', how = 'right')

city_merged

,Cities,Score,Rank,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Austin, TX",7.6,1.0,30.267153,-97.743061,3,Coffee Shop,Hotel,Cocktail Bar,Lounge,Bar,Italian Restaurant,Movie Theater,Nightclub,Burger Joint,Gay Bar
1,"Denver, CO",7.4,2.0,39.742043,-104.991531,3,Hotel,American Restaurant,Sandwich Place,Bar,Coffee Shop,Food Truck,Gym / Fitness Center,Lounge,Mexican Restaurant,Pizza Place
2,"Colorado Springs, CO",7.4,3.0,38.846127,-104.800644,5,Massage Studio,Gift Shop,Business Service,Yoga Studio,Fast Food Restaurant,Cosmetics Shop,Cycle Studio,Deli / Bodega,Dessert Shop,Discount Store
3,"Fayetteville, AR",7.3,4.0,36.082157,-94.171852,0,Pizza Place,Automotive Shop,Discount Store,Mexican Restaurant,Bookstore,Grocery Store,Farmers Market,Cosmetics Shop,Cycle Studio,Deli / Bodega
4,"Des Moines, IA",7.3,5.0,41.619549,-93.598022,2,University,Bagel Shop,Food Court,Cosmetics Shop,Cycle Studio,Deli / Bodega,Dessert Shop,Discount Store,Doctor's Office,Donut Shop
5,"San Juan, PR",3.2,125.0,18.466333,-66.105721,3,Plaza,Bar,Farmers Market,Historic Site,Sandwich Place,Caribbean Restaurant,Latin American Restaurant,Pier,Bakery,Music Venue
6,"Bakersfield, CA",5.3,124.0,35.393528,-119.043732,4,Shoe Store,Motorsports Shop,Yoga Studio,Farmers Market,Coffee Shop,Cosmetics Shop,Cycle Studio,Deli / Bodega,Dessert Shop,Discount Store
7,"Stockton, CA",5.4,123.0,37.961632,-121.275604,3,Fast Food Restaurant,Italian Restaurant,Seafood Restaurant,Grocery Store,Japanese Restaurant,Donut Shop,Mexican Restaurant,Doctor's Office,Deli / Bodega,Chinese Restaurant
8,"Shreveport, LA",5.4,122.0,32.523659,-93.763504,1,Business Service,Electronics Store,Moving Target,Yoga Studio,Cocktail Bar,Cosmetics Shop,Cycle Studio,Deli / Bodega,Dessert Shop,Discount Store
9,"Mobile, AL",5.5,121.0,30.695366,-88.039894,3,Sandwich Place,Bank,Seafood Restaurant,Hotel,Gym / Fitness Center,Asian Restaurant,Hotel Bar,Coffee Shop,Pizza Place,Plaza


In [29]:
latitude = 37.0902
longitude = -95.7129
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=3)
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
markers_colors = []
for lat, lon, poi, cluster in zip(city_merged['Latitude'], city_merged['Longitude'], city_merged['Cities'], city_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)

map_clusters

NameError: name 'folium' is not defined

In [30]:
city_new = df_score

city_new = city_new.join(city_grouped.set_index('Cities'), on='Cities', how = 'right')

city_new

,Cities,Score,Rank,Latitude,Longitude,American Restaurant,Arcade,Art Museum,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Beer Bar,Bookstore,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Burger Joint,Business Service,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Cosmetics Shop,Cycle Studio,Deli / Bodega,Dessert Shop,Discount Store,Doctor's Office,Donut Shop,Electronics Store,Farmers Market,Fast Food Restaurant,Food Court,Food Truck,Fried Chicken Joint,Gay Bar,Gift Shop,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Historic Site,Hotel,Hotel Bar,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Latin American Restaurant,Lounge,Marijuana Dispensary,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Motorsports Shop,Movie Theater,Moving Target,Music Venue,New American Restaurant,Nightclub,Noodle House,Optical Shop,Outdoor Sculpture,Paella Restaurant,Park,Pet Store,Pharmacy,Pier,Pizza Place,Plaza,Residential Building (Apartment / Condo),Restaurant,Rock Club,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Southern / Soul Food Restaurant,Souvenir Shop,Spa,Speakeasy,Steakhouse,Sushi Restaurant,Taco Place,Tea Room,Theater,Theme Park Ride / Attraction,Turkish Restaurant,University,Yoga Studio
0,"Austin, TX",7.6,1.0,30.267153,-97.743061,0.020000,0.000000,0.02,0.00,0.000000,0.0,0.01,0.0,0.000000,0.00,0.040000,0.01,0.00,0.000000,0.000000,0.010000,0.030000,0.000000,0.010000,0.010000,0.000000,0.000000,0.00,0.000000,0.010000,0.050000,0.080000,0.000000,0.01,0.000000,0.010000,0.0,0.00,0.01,0.000000,0.000000,0.00,0.01,0.000000,0.01,0.03,0.010000,0.000000,0.02,0.000000,0.010000,0.000000,0.080000,0.020000,0.01,0.030000,0.000000,0.01,0.030000,0.010000,0.050000,0.000000,0.000000,0.01,0.01,0.000000,0.0,0.030000,0.000000,0.020000,0.030000,0.030000,0.000000,0.01,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.010000,0.000000,0.000000,0.020000,0.02,0.01,0.010000,0.000000,0.000000,0.02,0.0,0.000000,0.00,0.00,0.000000,0.000000,0.02,0.030000,0.02,0.01,0.000000,0.010000,0.000000,0.01,0.0,0.01
1,"Denver, CO",7.4,2.0,39.742043,-104.991531,0.055556,0.013889,0.00,0.00,0.000000,0.0,0.00,0.0,0.013889,0.00,0.027778,0.00,0.00,0.013889,0.013889,0.013889,0.013889,0.000000,0.013889,0.013889,0.013889,0.000000,0.00,0.013889,0.013889,0.013889,0.027778,0.013889,0.00,0.013889,0.013889,0.0,0.00,0.00,0.000000,0.000000,0.00,0.00,0.027778,0.00,0.00,0.000000,0.013889,0.00,0.013889,0.027778,0.000000,0.138889,0.013889,0.00,0.027778,0.013889,0.00,0.013889,0.000000,0.027778,0.013889,0.000000,0.00,0.00,0.027778,0.0,0.013889,0.000000,0.000000,0.013889,0.013889,0.013889,0.00,0.013889,0.000000,0.013889,0.00,0.00,0.000000,0.027778,0.000000,0.013889,0.027778,0.00,0.00,0.013889,0.013889,0.041667,0.00,0.0,0.013889,0.00,0.00,0.013889,0.013889,0.00,0.013889,0.00,0.00,0.027778,0.013889,0.000000,0.00,0.0,0.00
2,"Colorado Springs, CO",7.4,3.0,38.846127,-104.800644,0.000000,0.000000,0.00,0.00,0.000000,0.0,0.00,0.0,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.00,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.00,0.333333,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.333333,0.00,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.0,0.00
3,"Fayetteville, AR",7.3,4.0,36.082157,-94.171852,0.000000,0.000000,0.00,0.00,0.000000,0.2,0.00,0.0,0.000000,0.00,0.000000,0.00,0.20,0.000000

In [31]:
city_analysis = city_new.drop('Latitude', axis=1)
city_analysis = city_analysis.drop('Longitude', axis=1)

city_analysis

,Cities,Score,Rank,American Restaurant,Arcade,Art Museum,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Beer Bar,Bookstore,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Burger Joint,Business Service,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Cosmetics Shop,Cycle Studio,Deli / Bodega,Dessert Shop,Discount Store,Doctor's Office,Donut Shop,Electronics Store,Farmers Market,Fast Food Restaurant,Food Court,Food Truck,Fried Chicken Joint,Gay Bar,Gift Shop,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Historic Site,Hotel,Hotel Bar,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Latin American Restaurant,Lounge,Marijuana Dispensary,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Motorsports Shop,Movie Theater,Moving Target,Music Venue,New American Restaurant,Nightclub,Noodle House,Optical Shop,Outdoor Sculpture,Paella Restaurant,Park,Pet Store,Pharmacy,Pier,Pizza Place,Plaza,Residential Building (Apartment / Condo),Restaurant,Rock Club,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Southern / Soul Food Restaurant,Souvenir Shop,Spa,Speakeasy,Steakhouse,Sushi Restaurant,Taco Place,Tea Room,Theater,Theme Park Ride / Attraction,Turkish Restaurant,University,Yoga Studio
0,"Austin, TX",7.6,1.0,0.020000,0.000000,0.02,0.00,0.000000,0.0,0.01,0.0,0.000000,0.00,0.040000,0.01,0.00,0.000000,0.000000,0.010000,0.030000,0.000000,0.010000,0.010000,0.000000,0.000000,0.00,0.000000,0.010000,0.050000,0.080000,0.000000,0.01,0.000000,0.010000,0.0,0.00,0.01,0.000000,0.000000,0.00,0.01,0.000000,0.01,0.03,0.010000,0.000000,0.02,0.000000,0.010000,0.000000,0.080000,0.020000,0.01,0.030000,0.000000,0.01,0.030000,0.010000,0.050000,0.000000,0.000000,0.01,0.01,0.000000,0.0,0.030000,0.000000,0.020000,0.030000,0.030000,0.000000,0.01,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.010000,0.000000,0.000000,0.020000,0.02,0.01,0.010000,0.000000,0.000000,0.02,0.0,0.000000,0.00,0.00,0.000000,0.000000,0.02,0.030000,0.02,0.01,0.000000,0.010000,0.000000,0.01,0.0,0.01
1,"Denver, CO",7.4,2.0,0.055556,0.013889,0.00,0.00,0.000000,0.0,0.00,0.0,0.013889,0.00,0.027778,0.00,0.00,0.013889,0.013889,0.013889,0.013889,0.000000,0.013889,0.013889,0.013889,0.000000,0.00,0.013889,0.013889,0.013889,0.027778,0.013889,0.00,0.013889,0.013889,0.0,0.00,0.00,0.000000,0.000000,0.00,0.00,0.027778,0.00,0.00,0.000000,0.013889,0.00,0.013889,0.027778,0.000000,0.138889,0.013889,0.00,0.027778,0.013889,0.00,0.013889,0.000000,0.027778,0.013889,0.000000,0.00,0.00,0.027778,0.0,0.013889,0.000000,0.000000,0.013889,0.013889,0.013889,0.00,0.013889,0.000000,0.013889,0.00,0.00,0.000000,0.027778,0.000000,0.013889,0.027778,0.00,0.00,0.013889,0.013889,0.041667,0.00,0.0,0.013889,0.00,0.00,0.013889,0.013889,0.00,0.013889,0.00,0.00,0.027778,0.013889,0.000000,0.00,0.0,0.00
2,"Colorado Springs, CO",7.4,3.0,0.000000,0.000000,0.00,0.00,0.000000,0.0,0.00,0.0,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.00,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.00,0.333333,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.333333,0.00,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.0,0.00
3,"Fayetteville, AR",7.3,4.0,0.000000,0.000000,0.00,0.00,0.000000,0.2,0.00,0.0,0.000000,0.00,0.000000,0.00,0.20,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,

In [33]:
city_analysis2 = city_analysis.drop([2,3,4,6,8], axis=0)


In [34]:
city_analysis3 = city_analysis2.drop(['Score','Rank'], axis =1 )
city_analysis3

,Cities,American Restaurant,Arcade,Art Museum,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Beer Bar,Bookstore,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Burger Joint,Business Service,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Cosmetics Shop,Cycle Studio,Deli / Bodega,Dessert Shop,Discount Store,Doctor's Office,Donut Shop,Electronics Store,Farmers Market,Fast Food Restaurant,Food Court,Food Truck,Fried Chicken Joint,Gay Bar,Gift Shop,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Historic Site,Hotel,Hotel Bar,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Latin American Restaurant,Lounge,Marijuana Dispensary,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Motorsports Shop,Movie Theater,Moving Target,Music Venue,New American Restaurant,Nightclub,Noodle House,Optical Shop,Outdoor Sculpture,Paella Restaurant,Park,Pet Store,Pharmacy,Pier,Pizza Place,Plaza,Residential Building (Apartment / Condo),Restaurant,Rock Club,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Southern / Soul Food Restaurant,Souvenir Shop,Spa,Speakeasy,Steakhouse,Sushi Restaurant,Taco Place,Tea Room,Theater,Theme Park Ride / Attraction,Turkish Restaurant,University,Yoga Studio
0,"Austin, TX",0.020000,0.000000,0.02,0.00,0.000000,0.0,0.01,0.0,0.000000,0.00,0.040000,0.01,0.00,0.000000,0.000000,0.010000,0.030000,0.00,0.010000,0.010000,0.000000,0.000000,0.00,0.000000,0.010000,0.050000,0.080000,0.000000,0.01,0.000000,0.010000,0.0,0.00,0.01,0.0,0.000000,0.00,0.01,0.000000,0.01,0.03,0.01,0.000000,0.02,0.000000,0.010000,0.000000,0.080000,0.020000,0.01,0.030000,0.000000,0.01,0.030000,0.010000,0.050000,0.000000,0.0,0.01,0.01,0.000000,0.0,0.030000,0.0,0.020000,0.030000,0.030000,0.000000,0.01,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.010000,0.000000,0.000000,0.020000,0.02,0.01,0.010000,0.000000,0.000000,0.02,0.0,0.000000,0.00,0.00,0.000000,0.000000,0.02,0.030000,0.02,0.01,0.000000,0.010000,0.000000,0.01,0.0,0.01
1,"Denver, CO",0.055556,0.013889,0.00,0.00,0.000000,0.0,0.00,0.0,0.013889,0.00,0.027778,0.00,0.00,0.013889,0.013889,0.013889,0.013889,0.00,0.013889,0.013889,0.013889,0.000000,0.00,0.013889,0.013889,0.013889,0.027778,0.013889,0.00,0.013889,0.013889,0.0,0.00,0.00,0.0,0.000000,0.00,0.00,0.027778,0.00,0.00,0.00,0.013889,0.00,0.013889,0.027778,0.000000,0.138889,0.013889,0.00,0.027778,0.013889,0.00,0.013889,0.000000,0.027778,0.013889,0.0,0.00,0.00,0.027778,0.0,0.013889,0.0,0.000000,0.013889,0.013889,0.013889,0.00,0.013889,0.000000,0.013889,0.00,0.00,0.000000,0.027778,0.000000,0.013889,0.027778,0.00,0.00,0.013889,0.013889,0.041667,0.00,0.0,0.013889,0.00,0.00,0.013889,0.013889,0.00,0.013889,0.00,0.00,0.027778,0.013889,0.000000,0.00,0.0,0.00
5,"San Juan, PR",0.000000,0.000000,0.00,0.00,0.058824,0.0,0.00,0.0,0.058824,0.00,0.117647,0.00,0.00,0.000000,0.000000,0.000000,0.058824,0.00,0.000000,0.000000,0.000000,0.058824,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.00,0.00,0.0,0.058824,0.00,0.00,0.000000,0.00,0.00,0.00,0.000000,0.00,0.058824,0.000000,0.058824,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.058824,0.000000,0.000000,0.0,0.00,0.00,0.000000,0.0,0.000000,0.0,0.058824,0.000000,0.000000,0.000000,0.00,0.000000,0.058824,0.000000,0.00,0.00,0.058824,0.000000,0.117647,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.058824,0.00,0.0,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.058824,0.00,0.0,0.00
7,"Stockton, CA",0.000000,0.000000,0.00,0.00,0.000000,0.0,0.00,0.0,0.000000,0.05,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.05,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.050000,0.000000,0.0,0.05,0.05,0.0,0.000000,0.25,0.00,0.000000,0.00,0.00,0.00,0.000000,0.05,0.000000,0.0

In [35]:
from sklearn.preprocessing import StandardScaler

X = city_analysis3.values[:,1:]
X = np.nan_to_num(X)
cluster_dataset = StandardScaler().fit_transform(X)
cluster_dataset

/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


array([[ 0.22576277, -0.5       ,  2.        , -0.5       , -0.5       ,
         0.        ,  2.        ,  0.        , -0.96787356, -0.78250805,
        -0.13000683,  2.        , -0.5       , -0.5       , -0.5       ,
        -0.18930704,  0.42784638, -0.5       ,  0.87337062, -0.18930704,
        -0.5       , -0.5       , -0.5       , -0.5       ,  0.87337062,
         1.92135677,  1.69971899, -0.5       ,  2.        , -0.65957023,
         0.87337062,  0.        , -0.5       , -0.10314212,  0.        ,
        -0.5       , -0.5964503 ,  2.        , -0.5       ,  2.        ,
         2.        ,  2.        , -0.5       ,  0.30618622, -0.96787356,
        -0.45198827, -0.5       ,  0.3791535 ,  0.3519242 ,  2.        ,
         0.43919631, -0.65957023,  2.        ,  1.78387853, -0.16468612,
         1.69624453, -0.5       ,  0.        ,  2.        ,  2.        ,
        -0.76604592,  0.        ,  1.78387853,  0.        ,  0.18510527,
         1.78387853,  1.78387853, -0.5       ,  2. 

In [36]:
num_clusters = 2

k_means = KMeans(init="k-means++", n_clusters=num_clusters, n_init=12)
k_means.fit(cluster_dataset)
labels = k_means.labels_

print(labels)


[0 0 1 1 1]


In [37]:
city_analysis2["Labels"] = labels
city_analysis2.head(5)

,Cities,Score,Rank,American Restaurant,Arcade,Art Museum,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Beer Bar,Bookstore,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Burger Joint,Business Service,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Cosmetics Shop,Cycle Studio,Deli / Bodega,Dessert Shop,Discount Store,Doctor's Office,Donut Shop,Electronics Store,Farmers Market,Fast Food Restaurant,Food Court,Food Truck,Fried Chicken Joint,Gay Bar,Gift Shop,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Historic Site,Hotel,Hotel Bar,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Latin American Restaurant,Lounge,Marijuana Dispensary,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Motorsports Shop,Movie Theater,Moving Target,Music Venue,New American Restaurant,Nightclub,Noodle House,Optical Shop,Outdoor Sculpture,Paella Restaurant,Park,Pet Store,Pharmacy,Pier,Pizza Place,Plaza,Residential Building (Apartment / Condo),Restaurant,Rock Club,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Southern / Soul Food Restaurant,Souvenir Shop,Spa,Speakeasy,Steakhouse,Sushi Restaurant,Taco Place,Tea Room,Theater,Theme Park Ride / Attraction,Turkish Restaurant,University,Yoga Studio,Labels
0,"Austin, TX",7.6,1.0,0.020000,0.000000,0.02,0.00,0.000000,0.0,0.01,0.0,0.000000,0.00,0.040000,0.01,0.00,0.000000,0.000000,0.010000,0.030000,0.00,0.010000,0.010000,0.000000,0.000000,0.00,0.000000,0.010000,0.050000,0.080000,0.000000,0.01,0.000000,0.010000,0.0,0.00,0.01,0.0,0.000000,0.00,0.01,0.000000,0.01,0.03,0.01,0.000000,0.02,0.000000,0.010000,0.000000,0.080000,0.020000,0.01,0.030000,0.000000,0.01,0.030000,0.010000,0.050000,0.000000,0.0,0.01,0.01,0.000000,0.0,0.030000,0.0,0.020000,0.030000,0.030000,0.000000,0.01,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.010000,0.000000,0.000000,0.020000,0.02,0.01,0.010000,0.000000,0.000000,0.02,0.0,0.000000,0.00,0.00,0.000000,0.000000,0.02,0.030000,0.02,0.01,0.000000,0.010000,0.000000,0.01,0.0,0.01,0
1,"Denver, CO",7.4,2.0,0.055556,0.013889,0.00,0.00,0.000000,0.0,0.00,0.0,0.013889,0.00,0.027778,0.00,0.00,0.013889,0.013889,0.013889,0.013889,0.00,0.013889,0.013889,0.013889,0.000000,0.00,0.013889,0.013889,0.013889,0.027778,0.013889,0.00,0.013889,0.013889,0.0,0.00,0.00,0.0,0.000000,0.00,0.00,0.027778,0.00,0.00,0.00,0.013889,0.00,0.013889,0.027778,0.000000,0.138889,0.013889,0.00,0.027778,0.013889,0.00,0.013889,0.000000,0.027778,0.013889,0.0,0.00,0.00,0.027778,0.0,0.013889,0.0,0.000000,0.013889,0.013889,0.013889,0.00,0.013889,0.000000,0.013889,0.00,0.00,0.000000,0.027778,0.000000,0.013889,0.027778,0.00,0.00,0.013889,0.013889,0.041667,0.00,0.0,0.013889,0.00,0.00,0.013889,0.013889,0.00,0.013889,0.00,0.00,0.027778,0.013889,0.000000,0.00,0.0,0.00,0
5,"San Juan, PR",3.2,125.0,0.000000,0.000000,0.00,0.00,0.058824,0.0,0.00,0.0,0.058824,0.00,0.117647,0.00,0.00,0.000000,0.000000,0.000000,0.058824,0.00,0.000000,0.000000,0.000000,0.058824,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.00,0.00,0.0,0.058824,0.00,0.00,0.000000,0.00,0.00,0.00,0.000000,0.00,0.058824,0.000000,0.058824,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.058824,0.000000,0.000000,0.0,0.00,0.00,0.000000,0.0,0.000000,0.0,0.058824,0.000000,0.000000,0.000000,0.00,0.000000,0.058824,0.000000,0.00,0.00,0.058824,0.000000,0.117647,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.058824,0.00,0.0,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.058824,0.00,0.0,0.00,1
7,"Stockton, CA",5.4,123.0,0.000000,0.000000,0.00,0.00,0.000000,0.0,0.00,0.0,0.000000,0.05,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.05,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.050000,0.000000,0.0,0.05,0.05,0.0,0.000000,

In [38]:
city_groupby = city_analysis2.groupby('Labels').mean()
city_groupby


,Score,Rank,American Restaurant,Arcade,Art Museum,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Beer Bar,Bookstore,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Burger Joint,Business Service,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Cosmetics Shop,Cycle Studio,Deli / Bodega,Dessert Shop,Discount Store,Doctor's Office,Donut Shop,Electronics Store,Farmers Market,Fast Food Restaurant,Food Court,Food Truck,Fried Chicken Joint,Gay Bar,Gift Shop,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Historic Site,Hotel,Hotel Bar,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Latin American Restaurant,Lounge,Marijuana Dispensary,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Motorsports Shop,Movie Theater,Moving Target,Music Venue,New American Restaurant,Nightclub,Noodle House,Optical Shop,Outdoor Sculpture,Paella Restaurant,Park,Pet Store,Pharmacy,Pier,Pizza Place,Plaza,Residential Building (Apartment / Condo),Restaurant,Rock Club,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Southern / Soul Food Restaurant,Souvenir Shop,Spa,Speakeasy,Steakhouse,Sushi Restaurant,Taco Place,Tea Room,Theater,Theme Park Ride / Attraction,Turkish Restaurant,University,Yoga Studio
Labels,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,7.5,1.5,0.037778,0.006944,0.01,0.000000,0.000000,0.0,0.005,0.0,0.006944,0.000000,0.033889,0.005,0.000000,0.006944,0.006944,0.011944,0.021944,0.000000,0.011944,0.011944,0.006944,0.000000,0.000000,0.006944,0.011944,0.031944,0.053889,0.006944,0.005,0.006944,0.011944,0.0,0.000000,0.005000,0.0,0.000000,0.000000,0.005,0.013889,0.005,0.015,0.005,0.006944,0.010000,0.006944,0.018889,0.000000,0.109444,0.016944,0.005,0.028889,0.006944,0.005,0.021944,0.005000,0.038889,0.006944,0.0,0.005,0.005,0.013889,0.0,0.021944,0.0,0.010000,0.021944,0.021944,0.006944,0.005,0.006944,0.000000,0.006944,0.000000,0.000000,0.000000,0.018889,0.000000,0.006944,0.023889,0.01,0.005,0.011944,0.006944,0.020833,0.010000,0.0,0.006944,0.000000,0.000000,0.006944,0.006944,0.01,0.021944,0.01,0.005,0.013889,0.011944,0.000000,0.005,0.0,0.005
1,4.7,123.0,0.000000,0.000000,0.00,0.013333,0.019608,0.0,0.000,0.0,0.032941,0.043333,0.052549,0.000,0.013333,0.000000,0.000000,0.013333,0.019608,0.013333,0.000000,0.013333,0.000000,0.019608,0.016667,0.000000,0.000000,0.000000,0.013333,0.000000,0.000,0.016667,0.000000,0.0,0.016667,0.016667,0.0,0.019608,0.096667,0.000,0.000000,0.000,0.000,0.000,0.000000,0.016667,0.032941,0.026667,0.019608,0.026667,0.013333,0.000,0.016667,0.016667,0.000,0.000000,0.019608,0.000000,0.000000,0.0,0.000,0.000,0.016667,0.0,0.000000,0.0,0.019608,0.000000,0.000000,0.000000,0.000,0.000000,0.019608,0.000000,0.016667,0.016667,0.019608,0.030000,0.052549,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.076275,0.043333,0.0,0.000000,0.016667,0.013333,0.000000,0.000000,0.00,0.000000,0.00,0.000,0.000000,0.000000,0.019608,0.000,0.0,0.000


In [45]:
city_corr = city_analysis2.corr()
city_corr


,Score,Rank,American Restaurant,Arcade,Art Museum,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Beer Bar,Bookstore,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Burger Joint,Business Service,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Cosmetics Shop,Cycle Studio,Deli / Bodega,Dessert Shop,Discount Store,Doctor's Office,Donut Shop,Electronics Store,Farmers Market,Fast Food Restaurant,Food Court,Food Truck,Fried Chicken Joint,Gay Bar,Gift Shop,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Historic Site,Hotel,Hotel Bar,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Latin American Restaurant,Lounge,Marijuana Dispensary,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Motorsports Shop,Movie Theater,Moving Target,Music Venue,New American Restaurant,Nightclub,Noodle House,Optical Shop,Outdoor Sculpture,Paella Restaurant,Park,Pet Store,Pharmacy,Pier,Pizza Place,Plaza,Residential Building (Apartment / Condo),Restaurant,Rock Club,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Southern / Soul Food Restaurant,Souvenir Shop,Spa,Speakeasy,Steakhouse,Sushi Restaurant,Taco Place,Tea Room,Theater,Theme Park Ride / Attraction,Turkish Restaurant,University,Yoga Studio,Labels
Score,1.000000,-0.866713,0.711956,0.493596,0.556076,-0.099969,-0.818494,NaN,0.556076,NaN,-0.777820,-0.175257,-0.657971,0.556076,-0.099969,0.493596,0.493596,0.229463,-0.445311,-0.099969,0.830604,0.229463,0.493596,-0.818494,-0.131209,0.493596,0.830604,0.715625,0.730482,0.493596,0.556076,0.006092,0.830604,NaN,-0.131209,-0.020622,NaN,-0.818494,-0.151379,0.556076,0.493596,0.556076,0.556076,0.556076,0.493596,0.093103,-0.777820,0.150369,-0.818494,0.787799,0.376797,0.556076,0.495819,0.006092,0.556076,0.791407,-0.745165,0.817772,0.493596,NaN,0.556076,0.556076,0.140854,NaN,0.791407,NaN,-0.647281,0.791407,0.791407,0.493596,0.556076,0.493596,-0.818494,0.493596,-0.131209,-0.131209,-0.818494,0.188660,-0.876662,0.493596,0.830604,0.556076,0.556076,0.830604,0.493596,-0.477669,-0.044364,NaN,0.493596,-0.131209,-0.099969,0.493596,0.493596,0.556076,0.791407,0.556076,0.556076,0.493596,0.830604,-0.818494,0.556076,NaN,0.556076,-0.857054
Rank,-0.866713,1.000000,-0.851679,-0.608026,-0.616425,0.391354,0.424947,NaN,-0.616425,NaN,0.553116,0.622586,0.246478,-0.616425,0.391354,-0.608026,-0.608026,0.028490,-0.035118,0.391354,-0.977295,0.028490,-0.608026,0.424947,0.408150,-0.608026,-0.977295,-0.810742,-0.681303,-0.608026,-0.616425,0.246999,-0.977295,NaN,0.408150,0.293816,NaN,0.424947,0.484119,-0.616425,-0.608026,-0.616425,-0.616425,-0.616425,-0.608026,0.164912,0.553116,0.110093,0.424947,-0.768377,-0.137991,-0.616425,-0.311537,0.246999,-0.616425,-0.905717,0.329531,-0.939824,-0.608026,NaN,-0.616425,-0.616425,0.069297,NaN,-0.905717,NaN,0.221471,-0.905717,-0.905717,-0.608026,-0.616425,-0.608026,0.424947,-0.608026,0.408150,0.408150,0.424947,0.281668,0.573834,-0.608026,-0.977295,-0.616425,-0.616425,-0.977295,-0.608026,0.694007,0.508489,NaN,-0.608026,0.408150,0.391354,-0.608026,-0.608026,-0.616425,-0.905717,-0.616425,-0.616425,-0.608026,-0.977295,0.424947,-0.616425,NaN,-0.616425,0.999760
American Restaurant,0.711956,-0.851679,1.000000,0.933837,0.112881,-0.348906,-0.348906,NaN,0.112881,NaN,-0.369420,-0.546044,-0.251055,0.112881,-0.348906,0.933837,0.933837,0.003885,-0.075413,-0.348906,0.943158,0.003885,0.933837,-0.348906,-0.348906,0.933837,0.943158,0.384332,0.283228,0.933837,0.112881,-0.092404,0.943158,NaN,-0.348906,-0.336584,NaN,-0.348906,-0.416210,0.112881,0.933837,0.112881,0.112881,0.112881,0.933837,-0.310017,-0.369420,-0.011254,-0.348906,0.831103,0.034270,0.112881,0.247174,-0.092404,0.112881,0.549949,-0.339373,0.622152,0.933837,NaN,0.112881,0.112881,0.167330,NaN,0.549949,NaN,-0.319334,0.549949,0.549949,0.933837,0.112881,0.933837,-0.348906,0.933837,-

In [46]:

indexNames = city_corr[city_corr['Score'] < 0.8].index

city_dropped = city_corr.drop(indexNames)

In [47]:
city_dropped


,Score,Rank,American Restaurant,Arcade,Art Museum,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Beer Bar,Bookstore,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Burger Joint,Business Service,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Cosmetics Shop,Cycle Studio,Deli / Bodega,Dessert Shop,Discount Store,Doctor's Office,Donut Shop,Electronics Store,Farmers Market,Fast Food Restaurant,Food Court,Food Truck,Fried Chicken Joint,Gay Bar,Gift Shop,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Historic Site,Hotel,Hotel Bar,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Latin American Restaurant,Lounge,Marijuana Dispensary,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Motorsports Shop,Movie Theater,Moving Target,Music Venue,New American Restaurant,Nightclub,Noodle House,Optical Shop,Outdoor Sculpture,Paella Restaurant,Park,Pet Store,Pharmacy,Pier,Pizza Place,Plaza,Residential Building (Apartment / Condo),Restaurant,Rock Club,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Southern / Soul Food Restaurant,Souvenir Shop,Spa,Speakeasy,Steakhouse,Sushi Restaurant,Taco Place,Tea Room,Theater,Theme Park Ride / Attraction,Turkish Restaurant,University,Yoga Studio,Labels
Score,1.000000,-0.866713,0.711956,0.493596,0.556076,-0.099969,-0.818494,NaN,0.556076,NaN,-0.777820,-0.175257,-0.657971,0.556076,-0.099969,0.493596,0.493596,0.229463,-0.445311,-0.099969,0.830604,0.229463,0.493596,-0.818494,-0.131209,0.493596,0.830604,0.715625,0.730482,0.493596,0.556076,0.006092,0.830604,NaN,-0.131209,-0.020622,NaN,-0.818494,-0.151379,0.556076,0.493596,0.556076,0.556076,0.556076,0.493596,0.093103,-0.777820,0.150369,-0.818494,0.787799,0.376797,0.556076,0.495819,0.006092,0.556076,0.791407,-0.745165,0.817772,0.493596,NaN,0.556076,0.556076,0.140854,NaN,0.791407,NaN,-0.647281,0.791407,0.791407,0.493596,0.556076,0.493596,-0.818494,0.493596,-0.131209,-0.131209,-0.818494,0.188660,-0.876662,0.493596,0.830604,0.556076,0.556076,0.830604,0.493596,-0.477669,-0.044364,NaN,0.493596,-0.131209,-0.099969,0.493596,0.493596,0.556076,0.791407,0.556076,0.556076,0.493596,0.830604,-0.818494,0.556076,NaN,0.556076,-0.857054
Automotive Shop,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Bagel Shop,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Café,0.830604,-0.977295,0.943158,0.761876,0.436685,-0.399521,-0.399521,NaN,0.436685,NaN,-0.496342,-0.625256,-0.249048,0.436685,-0.399521,0.761876,0.761876,-0.028142,0.003275,-0.399521,1.000000,-0.028142,0.761876,-0.399521,-0.399521,0.761876,1.000000,0.669304,0.540698,0.761876,0.436685,-0.193970,1.000000,NaN,-0.399521,-0.321993,NaN,-0.399521,-0.476588,0.436685,0.761876,0.436685,0.436685,0.436685,0.761876,-0.229483,-0.496342,-0.085780,-0.399521,0.815892,0.089884,0.436685,0.297243,-0.193970,0.436685,0.796262,-0.334811,0.846979,0.761876,NaN,0.436685,0.436685,0.023386,NaN,0.796262,NaN,-0.258168,0.796262,0.796262,0.761876,0.436685,0.761876,-0.399521,0.761876,-0.399521,-0.399521,-0.399521,-0.225779,-0.550542,0.761876,1.000000,0.436685,0.436685,1.000000,0.761876,-0.617

In [48]:
city_dropped.dropna(subset=['Score'], how='all', inplace = True)
city_dropped


,Score,Rank,American Restaurant,Arcade,Art Museum,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Beer Bar,Bookstore,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Burger Joint,Business Service,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Cosmetics Shop,Cycle Studio,Deli / Bodega,Dessert Shop,Discount Store,Doctor's Office,Donut Shop,Electronics Store,Farmers Market,Fast Food Restaurant,Food Court,Food Truck,Fried Chicken Joint,Gay Bar,Gift Shop,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Historic Site,Hotel,Hotel Bar,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Latin American Restaurant,Lounge,Marijuana Dispensary,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Motorsports Shop,Movie Theater,Moving Target,Music Venue,New American Restaurant,Nightclub,Noodle House,Optical Shop,Outdoor Sculpture,Paella Restaurant,Park,Pet Store,Pharmacy,Pier,Pizza Place,Plaza,Residential Building (Apartment / Condo),Restaurant,Rock Club,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Southern / Soul Food Restaurant,Souvenir Shop,Spa,Speakeasy,Steakhouse,Sushi Restaurant,Taco Place,Tea Room,Theater,Theme Park Ride / Attraction,Turkish Restaurant,University,Yoga Studio,Labels
Score,1.000000,-0.866713,0.711956,0.493596,0.556076,-0.099969,-0.818494,NaN,0.556076,NaN,-0.777820,-0.175257,-0.657971,0.556076,-0.099969,0.493596,0.493596,0.229463,-0.445311,-0.099969,0.830604,0.229463,0.493596,-0.818494,-0.131209,0.493596,0.830604,0.715625,0.730482,0.493596,0.556076,0.006092,0.830604,NaN,-0.131209,-0.020622,NaN,-0.818494,-0.151379,0.556076,0.493596,0.556076,0.556076,0.556076,0.493596,0.093103,-0.777820,0.150369,-0.818494,0.787799,0.376797,0.556076,0.495819,0.006092,0.556076,0.791407,-0.745165,0.817772,0.493596,NaN,0.556076,0.556076,0.140854,NaN,0.791407,NaN,-0.647281,0.791407,0.791407,0.493596,0.556076,0.493596,-0.818494,0.493596,-0.131209,-0.131209,-0.818494,0.188660,-0.876662,0.493596,0.830604,0.556076,0.556076,0.830604,0.493596,-0.477669,-0.044364,NaN,0.493596,-0.131209,-0.099969,0.493596,0.493596,0.556076,0.791407,0.556076,0.556076,0.493596,0.830604,-0.818494,0.556076,NaN,0.556076,-0.857054
Café,0.830604,-0.977295,0.943158,0.761876,0.436685,-0.399521,-0.399521,NaN,0.436685,NaN,-0.496342,-0.625256,-0.249048,0.436685,-0.399521,0.761876,0.761876,-0.028142,0.003275,-0.399521,1.000000,-0.028142,0.761876,-0.399521,-0.399521,0.761876,1.000000,0.669304,0.540698,0.761876,0.436685,-0.193970,1.000000,NaN,-0.399521,-0.321993,NaN,-0.399521,-0.476588,0.436685,0.761876,0.436685,0.436685,0.436685,0.761876,-0.229483,-0.496342,-0.085780,-0.399521,0.815892,0.089884,0.436685,0.297243,-0.193970,0.436685,0.796262,-0.334811,0.846979,0.761876,NaN,0.436685,0.436685,0.023386,NaN,0.796262,NaN,-0.258168,0.796262,0.796262,0.761876,0.436685,0.761876,-0.399521,0.761876,-0.399521,-0.399521,-0.399521,-0.225779,-0.550542,0.761876,1.000000,0.436685,0.436685,1.000000,0.761876,-0.617832,-0.557761,NaN,0.761876,-0.399521,-0.399521,0.761876,0.761876,0.436685,0.796262,0.436685,0.436685,0.761876,1.000000,-0.399521,0.436685,NaN,0.436685,-0.978622
Clothing Store,0.830604,-0.977295,0.943158,0.761876,0.436685,-0.399521,-0.399521,NaN,0.436685,NaN,-0.496342,-0.625256,-0.249048,0.436685,-0.399521,0.761876,0.761876,-0.028142,0.003275,-0.399521,1.000000,-0.028142,0.761876,-0.399521,-0.399521,0.761876,1.000000,0.669304,0.540698,0.761876,0.436685,-0.193970,1.000000,NaN,-0.399521,-0.321993,NaN,-0.399521,-0.476588,0.436685,0.761876,0.436685,0.436685,0.436685,0.761876,-0.229483,-0.496342,-0.085780,-0.399521,0.815892,0.089884,0.436685,0.297243,-0.193970,0.436685,0.796262,-0.334811,0.846979,0.761876,NaN,0.436685,0.436685,0.023386,NaN,0.796262,NaN,-0.258168,0.796262,0.796262,0.761876,0.436685,0.761876,-0.399521,0.761876,-0.399521,-0.399521,-0.3995

In [ ]:
city_df = city_analysis2[['Score','Café','Clothing Store','Dessert Shop','Restaurant','Salad Place','Theater']]
city_df